In [1]:
###### Question 1

import numpy as np
import pandas as pd

# import datasets
meter_list = pd.read_excel('/Users/hanaelhassani/Desktop/Jake/Gorilla/gorilla_test_data.xlsx', sheet_name='meter_list')

forecast = pd.read_excel('/Users/hanaelhassani/Desktop/Jake/Gorilla/gorilla_test_data.xlsx', sheet_name='forecast_table')

rate = pd.read_excel('/Users/hanaelhassani/Desktop/Jake/Gorilla/gorilla_test_data.xlsx', sheet_name='rate_table')


# Gather data into one table
full = forecast.merge(meter_list, left_on='meter_id', right_on='meter_id')


# Round aq to lower bound
full.loc[full['aq_kwh'] < 73200, 'aq_min_kwh'] = 0
full.loc[(full['aq_kwh'] >= 73200) & (full['aq_kwh'] < 732000), 'aq_min_kwh'] = 73200
full.loc[full['aq_kwh'] >= 732000, 'aq_min_kwh'] = 732000


# Create column showing which date range an entry is in
full.loc[full['date'] < '2020-10-01', 'date_range'] = '2020-04-01'
full.loc[(full['date'] >= '2020-10-01') & (full['date'] < '2021-04-01'), 'date_range'] = '2020-10-01'
full.loc[(full['date'] >= '2021-04-01') & (full['date'] < '2021-10-01'), 'date_range'] = '2021-04-01'
full.loc[(full['date'] >= '2021-10-01') & (full['date'] < '2022-04-01'), 'date_range'] = '2021-10-01'
full.loc[(full['date'] >= '2022-04-01') & (full['date'] < '2022-10-01'), 'date_range'] = '2022-04-01'
full.loc[(full['date'] >= '2022-10-01') & (full['date'] < '2023-04-01'), 'date_range'] = '2022-10-01'
full.loc[(full['date'] >= '2023-04-01') & (full['date'] < '2023-10-01'), 'date_range'] = '2023-04-01'
full.loc[(full['date'] >= '2023-10-01') & (full['date'] < '2024-04-01'), 'date_range'] = '2023-10-01'
full.loc[(full['date'] >= '2024-04-01') & (full['date'] < '2024-10-01'), 'date_range'] = '2024-04-01'
full.loc[(full['date'] >= '2024-10-01') & (full['date'] < '2025-04-01'), 'date_range'] = '2024-10-01'


# rename columns to match rate table
full = full.rename(columns={'date': 'day', 'date_range': 'date'})
full['date']= pd.to_datetime(full['date'])


# collate all data so rate matches each entry
df = pd.merge(rate, full)


# Find pence per day
df['p_per_day'] = df['kwh'] * df['rate_p_per_kwh']


# Convert pence to pounds
df['£_per_day'] = df.loc[:, 'p_per_day']
df['£_per_day'] = df['£_per_day']/100


# Sum required values based on meter id
total = df.groupby('meter_id')['£_per_day', 'kwh'].sum()
total = total.round(2)
print(total)

          £_per_day       kwh
meter_id                     
14676236     100.15   28978.0
34509937     275.49   78324.0
50264822     731.24  265667.0
88357331    1433.16  484399.0


/var/folders/hz/jhlbg8892451n3lcy0h7h3wr0000gn/T/ipykernel_29928/3030637712.py:56: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  total = df.groupby('meter_id')['£_per_day', 'kwh'].sum()


In [8]:
##### Question 2

import random

def random_meter_list():
    meter_list = pd.DataFrame()
    meter_list['meter_id'] = np.random.randint(100000000000, 999999999999, size=random.randint(1,10))
    meter_list['exit_zone'] = 'EA1'
    meter_list['aq_kwh'] = np.random.randint(5000,500000, size=len(meter_list))

    return(meter_list)

In [9]:
random_meter_list()

,meter_id,exit_zone,aq_kwh
0,938803665829,EA1,351959
1,443254502040,EA1,289340
2,988812238369,EA1,338225
3,150713649360,EA1,492145
4,564256193760,EA1,318589
5,554565088066,EA1,155850
6,243287113132,EA1,248471
7,496925694194,EA1,242810


In [10]:
##### Question 3

def random_forecast(random_meter_list, start_date, duration):
    
    
    # meter list variable used in Q4
    meter_list = random_meter_list()
    forecast = meter_list
    
    
    # Remove extra data, keep meter id
    forecast = forecast.drop(columns=['exit_zone', 'aq_kwh'])
    
    
    # Set dates
    datelist = pd.date_range(start=start_date, periods=duration, freq='1d')
    print(datelist)
    
    
    # Match number of rows in dataframe to dates
    forecast = pd.DataFrame({'meter_id':forecast['meter_id'].unique().tolist()*len(datelist), 'date':np.repeat(datelist,len(forecast['meter_id'].unique()))})

    
    # Add forecast data
    forecast['kwh'] = np.random.randint(1,1000, size=len(forecast))
    return(meter_list, forecast)

In [11]:
# Test
random_forecast(random_meter_list, '2021-01-01', 365)

DatetimeIndex(['2021-01-01', '2021-01-02', '2021-01-03', '2021-01-04',
               '2021-01-05', '2021-01-06', '2021-01-07', '2021-01-08',
               '2021-01-09', '2021-01-10',
               ...
               '2021-12-22', '2021-12-23', '2021-12-24', '2021-12-25',
               '2021-12-26', '2021-12-27', '2021-12-28', '2021-12-29',
               '2021-12-30', '2021-12-31'],
              dtype='datetime64[ns]', length=365, freq='D')


(       meter_id exit_zone  aq_kwh
 0  471176303586       EA1  139138
 1  370933141603       EA1  480643
 2  173536345139       EA1  313695
 3  727477486451       EA1  425307
 4  509750959850       EA1  486595
 5  605070305301       EA1   32499,
           meter_id       date  kwh
 0     471176303586 2021-01-01  866
 1     370933141603 2021-01-01  927
 2     173536345139 2021-01-01   41
 3     727477486451 2021-01-01  214
 4     509750959850 2021-01-01  223
 ...            ...        ...  ...
 2185  370933141603 2021-12-31   96
 2186  173536345139 2021-12-31  790
 2187  727477486451 2021-12-31  506
 2188  509750959850 2021-12-31  398
 2189  605070305301 2021-12-31  982
 
 [2190 rows x 3 columns])

In [12]:
##### Question 4

import time

def prices(random_forecast, start_date, duration):
    time1 = time.process_time()
    # import datasets
    meter_list, forecast = random_forecast(random_meter_list, start_date, duration)
    print(meter_list)
    print(forecast)

    rate = pd.read_excel('/Users/hanaelhassani/Desktop/Jake/Gorilla/gorilla_test_data.xlsx', sheet_name='rate_table')


    # Gather data into one table
    full = forecast.merge(meter_list, left_on='meter_id', right_on='meter_id')
    print(full)

    # Round aq to lower bound
    full.loc[full['aq_kwh'] < 73200, 'aq_min_kwh'] = 0
    full.loc[(full['aq_kwh'] >= 73200) & (full['aq_kwh'] < 732000), 'aq_min_kwh'] = 73200
    full.loc[full['aq_kwh'] >= 732000, 'aq_min_kwh'] = 732000
    

    # Create column showing which date range an entry is in
    full.loc[full['date'] < '2020-10-01', 'date_range'] = '2020-04-01'
    full.loc[(full['date'] >= '2020-10-01') & (full['date'] < '2021-04-01'), 'date_range'] = '2020-10-01'
    full.loc[(full['date'] >= '2021-04-01') & (full['date'] < '2021-10-01'), 'date_range'] = '2021-04-01'
    full.loc[(full['date'] >= '2021-10-01') & (full['date'] < '2022-04-01'), 'date_range'] = '2021-10-01'
    full.loc[(full['date'] >= '2022-04-01') & (full['date'] < '2022-10-01'), 'date_range'] = '2022-04-01'
    full.loc[(full['date'] >= '2022-10-01') & (full['date'] < '2023-04-01'), 'date_range'] = '2022-10-01'
    full.loc[(full['date'] >= '2023-04-01') & (full['date'] < '2023-10-01'), 'date_range'] = '2023-04-01'
    full.loc[(full['date'] >= '2023-10-01') & (full['date'] < '2024-04-01'), 'date_range'] = '2023-10-01'
    full.loc[(full['date'] >= '2024-04-01') & (full['date'] < '2024-10-01'), 'date_range'] = '2024-04-01'
    full.loc[(full['date'] >= '2024-10-01') & (full['date'] < '2025-04-01'), 'date_range'] = '2024-10-01'


    # rename columns to match rate table
    full = full.rename(columns={'date': 'day', 'date_range': 'date'})
    full['date']= pd.to_datetime(full['date'])


    # collate all data so rate matches each entry
    df = pd.merge(rate, full)


    # Find pence per day
    df['p_per_day'] = df['kwh'] * df['rate_p_per_kwh']


    # Convert pence to pounds
    df['£_per_day'] = df.loc[:, 'p_per_day']
    df['£_per_day'] = df['£_per_day']/100


    # Sum required values based on meter id
    total = df.groupby('meter_id')['£_per_day', 'kwh'].sum()
    total = total.round(2)
    time2 = time.process_time()
    total_time = time2 - time1
    print('Calculation took',total_time,'seconds')
    return(total)

In [14]:
prices(random_forecast, '2023-01-01', 3650)

DatetimeIndex(['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04',
               '2023-01-05', '2023-01-06', '2023-01-07', '2023-01-08',
               '2023-01-09', '2023-01-10',
               ...
               '2032-12-19', '2032-12-20', '2032-12-21', '2032-12-22',
               '2032-12-23', '2032-12-24', '2032-12-25', '2032-12-26',
               '2032-12-27', '2032-12-28'],
              dtype='datetime64[ns]', length=3650, freq='D')
       meter_id exit_zone  aq_kwh
0  871105923401       EA1  440897
          meter_id       date  kwh
0     871105923401 2023-01-01  396
1     871105923401 2023-01-02  775
2     871105923401 2023-01-03  345
3     871105923401 2023-01-04  149
4     871105923401 2023-01-05  649
...            ...        ...  ...
3645  871105923401 2032-12-24  678
3646  871105923401 2032-12-25  887
3647  871105923401 2032-12-26  387
3648  871105923401 2032-12-27  946
3649  871105923401 2032-12-28  502

[3650 rows x 3 columns]
          meter_id       date  kwh e

/var/folders/hz/jhlbg8892451n3lcy0h7h3wr0000gn/T/ipykernel_29928/3669405355.py:57: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  total = df.groupby('meter_id')['£_per_day', 'kwh'].sum()


,£_per_day,kwh
meter_id,,
871105923401,1858.6,409614


In [15]:
# Using the time funcion, the calculation time increases from around 0.2 seconds for one meter over a year duration to 0.3 seconds for 10 years.
# The increase is around 30%.
# Adding more meters increases the time exponentially, with 1000 meters taking around 30 seconds for one year
# Taking almost 1000 meters for 10 years takes around 2 minutes.


# I believe the calculation can be refined by reducing the number of lines of code.
# This can be done by using shortcuts and functions that I'm unaware of at this point.
# It can be tested using a more accurate benchmarking procedure and can be evaluated line by line to find where the code is slowest.

# There are no loops being used but I can admit the code is quite clunky and parts can be re-written using more efficient techniques.
# I would start to improve it by looking at the dataframe manipulation as I think there are probably better ways to do what I have done
# # However, my first step to improve the code would be analysing the speed line by line on a large dataset
